In [1]:
!pip install pandas scikit-learn matplotlib seaborn transformers torch sentence-transformers

/Users/amirthavarshanimahadevan/Documents/GSU/Fundamental of DS/Project/FDS_Project/projectEnv/bin/pip: line 2: /Users/amirthavarshanimahadevan/Documents/GSU/Fundamental of DS/Project/projectEnv/bin/python: No such file or directory
/Users/amirthavarshanimahadevan/Documents/GSU/Fundamental of DS/Project/FDS_Project/projectEnv/bin/pip: line 2: exec: /Users/amirthavarshanimahadevan/Documents/GSU/Fundamental of DS/Project/projectEnv/bin/python: cannot execute: No such file or directory


In [2]:
import pandas as pd

import numpy as np
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

/Users/amirthavarshanimahadevan/Documents/GSU/Fundamental of DS/Project/FDS_Project/projectEnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#load the dataset

df = pd.read_csv('Data/train.csv')
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


# pre-processing 

In [4]:
# Missing Data Analysis
missing_data = df.isnull().sum()
missing_percentage = (missing_data /df.shape[0]) * 100

# Create a DataFrame using the two arrays
missing_df = pd.DataFrame({
    'Missing Data': missing_data,
    'Missing Percentage': missing_percentage
}, index=df.columns)

# Filter to include only rows with non-zero missing data
missing_df_filtered = missing_df[missing_df['Missing Data'] > 0]

# Display the filtered DataFrame
missing_df_filtered

,Missing Data,Missing Percentage
keyword,61,0.801261
location,2533,33.272035


In [32]:
#drop 'location'
clean_df = df.drop(columns='location')

In [33]:
# Drop rows where 'keyword' has null values
df_cleaned = clean_df.dropna(subset=['keyword'])
df_cleaned

,id,keyword,text,target
31,48,ablaze,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,Crying out for more! Set me ablaze,0
35,53,ablaze,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...
7578,10830,wrecked,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,Three days off from work and they've pretty mu...,0
7580,10832,wrecked,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,@engineshed Great atmosphere at the British Li...,0


# Word Embedding

## 1. BERT embedding

In [ ]:
# Load BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Function to compute BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Average the last hidden state to get a fixed-length vector
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

# Apply the BERT embedding function to both 'text' and 'keyword' columns
df_cleaned['text_embedding'] = df_cleaned['text'].apply(get_bert_embedding)
df_cleaned['keyword_embedding'] = df_cleaned['keyword'].apply(get_bert_embedding)

# Combine embeddings into a single feature matrix
X = np.hstack([np.stack(df_cleaned['text_embedding'].values), np.stack(df_cleaned['keyword_embedding'].values)])
y = df_cleaned['target'].values  # Assuming 'target' is your label column


# Step 3: Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Train a simple classifier, e.g., Random Forest
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

/var/folders/mh/_hw9vphd3kd8psnxbg4f7dtm0000gn/T/ipykernel_48256/110014366.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['text_embedding'] = df_cleaned['text'].apply(get_bert_embedding)
/var/folders/mh/_hw9vphd3kd8psnxbg4f7dtm0000gn/T/ipykernel_48256/110014366.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['keyword_embedding'] = df_cleaned['keyword'].apply(get_bert_embedding)


              precision    recall  f1-score   support

           0       0.80      0.90      0.84      1302
           1       0.83      0.69      0.75       964

    accuracy                           0.81      2266
   macro avg       0.81      0.79      0.80      2266
weighted avg       0.81      0.81      0.80      2266



## 2. SBERT embedding

In [30]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Calculate embeddings for 'text' and 'keyword'
text_embeddings = model.encode(df_cleaned['text'].tolist())  # Convert 'text' to a list
df_cleaned['keyword_embeddings'] = df_cleaned['keyword'].apply(lambda x: model.encode(x))
print(f"Embeddings shape: {text_embeddings.shape}")


# 3. Calculate cosine similarity between text embeddings
similarities = cosine_similarity(text_embeddings)

print(f"Text Embeddings Shape: {text_embeddings.shape}")
print("\nCosine Similarities Matrix:\n")
print(similarities)


Embeddings shape: (7552, 384)
Text Embeddings Shape: (7552, 384)

Cosine Similarities Matrix:

[[1.0000001  0.17108253 0.37499806 ... 0.3040543  0.28843108 0.2948931 ]
 [0.17108253 0.99999994 0.21963425 ... 0.2392297  0.24595247 0.06717228]
 [0.37499806 0.21963425 1.         ... 0.19039425 0.30009043 0.17236699]
 ...
 [0.3040543  0.2392297  0.19039425 ... 1.0000005  0.28146535 0.8141388 ]
 [0.28843108 0.24595247 0.30009043 ... 0.28146535 1.0000001  0.27518782]
 [0.2948931  0.06717228 0.17236699 ... 0.8141388  0.27518782 1.0000001 ]]


/var/folders/mh/_hw9vphd3kd8psnxbg4f7dtm0000gn/T/ipykernel_48256/1661363783.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['keyword_embeddings'] = df_cleaned['keyword'].apply(lambda x: model.encode(x))


In [11]:
# Convert 'keyword_embeddings' to a 2D NumPy array
keyword_embeddings = np.vstack(df_cleaned['keyword_embeddings'].values)

# Combine embeddings into a single feature matrix
X = np.hstack((text_embeddings, keyword_embeddings))
y = df_cleaned['target'].values  # Assuming 'target' is your label column
#X = embeddings
#y = df_cleaned['target'].values  # Assuming 'target' is your label column


# Step 3: Split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Train a simple classifier, e.g., Random Forest
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

print("Accuracy : ",accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84      1302
           1       0.83      0.70      0.76       964

    accuracy                           0.81      2266
   macro avg       0.82      0.80      0.80      2266
weighted avg       0.81      0.81      0.81      2266

Accuracy :  0.8111209179170344


# Model Building

In [12]:
# function for experiment

def experiment(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    #print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred), model

## 1. Random Forest Classifier

In [13]:
n_estimators_list = [2, 20, 100, 200, 250] # number of decision trees

accuracy_list = []
for ne in n_estimators_list:
    (accuracy, model) = experiment(RandomForestClassifier(n_estimators=ne), X, y)

    accuracy_list.append(accuracy)
        
    print("\nModel : ", model)
    print("Accuracy : ", accuracy)
    print("--------------------------------------------------------")



Model :  RandomForestClassifier(n_estimators=2)
Accuracy :  0.6972639011473963
--------------------------------------------------------

Model :  RandomForestClassifier(n_estimators=20)
Accuracy :  0.7978817299205648
--------------------------------------------------------

Model :  RandomForestClassifier()
Accuracy :  0.8093556928508385
--------------------------------------------------------

Model :  RandomForestClassifier(n_estimators=200)
Accuracy :  0.8168578993821712
--------------------------------------------------------

Model :  RandomForestClassifier(n_estimators=250)
Accuracy :  0.8137687555163283
--------------------------------------------------------


## Model :  RandomForestClassifier(n_estimators=200)
## Accuracy :  0.8190644307149162

# --------------------------

## 2. Decision Tree Classifier

In [14]:
max_depth1 = [3,5,7,10,15,20]

for depth in max_depth1:
        
    (accuracy, model) = experiment(DecisionTreeClassifier(max_depth= depth), X, y)
    print("\nModel : ", model)
    print("max_depth : ", depth)
    print("Accuracy : ", accuracy)
    print("--------------------------------------------------------")


Model :  DecisionTreeClassifier(max_depth=3)
max_depth :  3
Accuracy :  0.6844660194174758
--------------------------------------------------------

Model :  DecisionTreeClassifier(max_depth=5)
max_depth :  5
Accuracy :  0.7206531332744925
--------------------------------------------------------

Model :  DecisionTreeClassifier(max_depth=7)
max_depth :  7
Accuracy :  0.732568402471315
--------------------------------------------------------

Model :  DecisionTreeClassifier(max_depth=10)
max_depth :  10
Accuracy :  0.7259488084730803
--------------------------------------------------------

Model :  DecisionTreeClassifier(max_depth=15)
max_depth :  15
Accuracy :  0.7038834951456311
--------------------------------------------------------

Model :  DecisionTreeClassifier(max_depth=20)
max_depth :  20
Accuracy :  0.6897616946160635
--------------------------------------------------------


## 3. SVM 

In [28]:
svm_kernel= ['poly','linear','rbf','sigmoid']

for svm_ke in svm_kernel:
    svm = SVC(C=1, kernel=svm_ke, gamma='scale')  # Linear kernel for binary classification
    svm.fit(X_train, y_train)

    # Predict
    y_pred = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

   
    print("\nkernel :", svm_ke)
    print(f"Accuracy : {accuracy}")
    print("\n--------------------------------------------------------")



kernel : poly
Accuracy : 0.8278905560458959

--------------------------------------------------------

kernel : linear
Accuracy : 0.8265666372462489

--------------------------------------------------------

kernel : rbf
Accuracy : 0.8305383936451898

--------------------------------------------------------

kernel : sigmoid
Accuracy : 0.8080317740511915

--------------------------------------------------------


## svm = SVC(C=1, kernel='rbf', gamma='scale') 
## Accuracy: 0.8305383936451898

## 4. Neural Network

In [29]:
nn_activation = ['relu','identity','logistic','tanh']
# Step 2: Initialize and train your neural network
# Multi-Layer Perceptron (MLP) 
for nn_act in nn_activation:
    model = MLPClassifier(hidden_layer_sizes=(10,), activation='tanh', max_iter=10000, random_state=42)  
    model.fit(X_train, y_train)

    # Step 3: Make predictions
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    print("\nActivation :", nn_act)
    print(f"Accuracy : {accuracy}")
    print("\n--------------------------------------------------------")


Activation : relu
Accuracy : 0.7784642541924095

--------------------------------------------------------

Activation : identity
Accuracy : 0.7784642541924095

--------------------------------------------------------

Activation : logistic
Accuracy : 0.7784642541924095

--------------------------------------------------------

Activation : tanh
Accuracy : 0.7784642541924095

--------------------------------------------------------


In [21]:
test_df = pd.read_csv('Data/test.csv')

clean_df = df.drop(columns='location')
clean_test_df = clean_df.dropna(subset=['keyword'])

In [22]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Calculate embeddings for 'text' and 'keyword'
text_embeddings = model.encode(clean_test_df['text'].tolist())  # Convert 'text' to a list
clean_test_df['keyword_embeddings'] = clean_test_df['keyword'].apply(lambda x: model.encode(x))
print(f"Embeddings shape: {text_embeddings.shape}")


# 3. Calculate cosine similarity between text embeddings
similarities = cosine_similarity(text_embeddings)

print(f"Text Embeddings Shape: {text_embeddings.shape}")
print("Cosine Similarities Matrix:")
print(similarities)

Embeddings shape: (7552, 384)
Text Embeddings Shape: (7552, 384)
Cosine Similarities Matrix:
[[1.0000001  0.17108253 0.37499806 ... 0.3040543  0.28843108 0.2948931 ]
 [0.17108253 0.99999994 0.21963425 ... 0.2392297  0.24595247 0.06717228]
 [0.37499806 0.21963425 1.         ... 0.19039425 0.30009043 0.17236699]
 ...
 [0.3040543  0.2392297  0.19039425 ... 1.0000005  0.28146535 0.8141388 ]
 [0.28843108 0.24595247 0.30009043 ... 0.28146535 1.0000001  0.27518782]
 [0.2948931  0.06717228 0.17236699 ... 0.8141388  0.27518782 1.0000001 ]]


/var/folders/mh/_hw9vphd3kd8psnxbg4f7dtm0000gn/T/ipykernel_48256/659444851.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_test_df['keyword_embeddings'] = clean_test_df['keyword'].apply(lambda x: model.encode(x))
